<em>Mixture</em> is the primary component of the PyChemkin framework. In addition to getting mixture thermodynamic and transport properties, such as density, heat capacity, and viscosity, you can combine two mixtures, find the equilibrium state of a mixture, or use a mixture to define the initial state of a reactor. In PyChemkin, a reactor model is seen as a black-box that transforms a mixture from its initial state to a new one.  

Illustration of basic <i><u>operations</u></i> available for the <em>Mixture</em> object in PyChemkin: <b>create</b>, <b>combine/mix</b>, <b>equilibrium</b>, and <b>process</b> (by a reactor model)
<img src="./mixture_concept.png" alt="Mixture Concept" width=900 height=auto>

This tutorial demonstrates different ways to <strong>create a mixture object</strong> in PyChemkin. The <i><u>recipe</u></i> method allows you to provide just the non-zero species components by using a list of species-fraction pairing tuples. The <i><u>array</u></i> method lets you use a mole/mass fraction array (for all species) to set up the mixture composition. The <i><u>equivalence ratio</u></i> method will create a new mixture from predefined <i>fuel</i> and <i>oxidizer</i> mixtures when you specify an equivalnce ratio value.

In [ ]:
import os
import chemkin as ck                      # Chemkin
import numpy as np                        # number crunching
import copy

# check working directory
current_dir = os.getcwd()
print("current working directory: " + current_dir)
# set verbose mode
ck.verbose = True

<h5>Step one</h5>The first step is always to create and preprocess the <em>Chemistry</em> set object for the current PyChemkin project. Here the GRI 3.0 mechanism for methane combustion is used to instantiate the <code>MyGasMech</code> <em>Chemistry</em> object.

In [ ]:
# set mechanism directory (the default chemkin mechanism data directory)
data_dir = os.path.join(ck.ansys_dir, "reaction", "data")
mechanism_dir = data_dir
# create a chemistry set based on GRI 3.0
MyGasMech = ck.Chemistry(label="GRI 3.0")
# set mechanism input files
# inclusion of the full file path is recommended
MyGasMech.chemfile = os.path.join(mechanism_dir, "grimech30_chem.inp")
MyGasMech.thermfile = os.path.join(mechanism_dir, "grimech30_thermo.dat")
MyGasMech.tranfile = os.path.join(mechanism_dir, "grimech30_transport.dat")
# preprocess the mechanism files
iError = MyGasMech.preprocess()

<hr>Note: You can find the detail information of any <strong>PyChemkin</strong> method by printing its <code>doc</code> string. For example, to learn more about the <em>Mixture</em> <code>listcomposition</code> method, you can type the following print command at the Python prompt. 

In [ ]:
print(ck.Mixture.listcomposition.__doc__)

<h5>Step two</h5>Create a new <em>Mixture</em> object <code>premixed</code> by associating it with the <code>MyGasMech</code> chemistry set. That is, the <code>premixed</code> mixture inherits all <code>MyGasMech</code>'s attributes such as number of species, reactions, and property data.  

In [ ]:
# create a mixture called premixed based on the MyGasMech chemistry set
premixed = ck.Mixture(MyGasMech)

<h5>Step three</h5>Specify the mixture conditions. Mixture pressure [dynes/cm<sup>2</sup>] and temperature[K] should be provided to properly set up the <code>premixed</code> mixture.

In [ ]:
# set mixture pressure [dynes/cm2]
mixpressure = 2.0                                # given in atm
# convert to dynes/cm2
premixed.pressure = mixpressure * ck.Patm
# set mixture temperature [K]
premixed.temperature = 500.0

Here the <i><u>recipe</u></i> method is used to define the composition of <code>premixed</code>. The composition <i>"recipe"</i> is a list of tuples of species symbol and fraction pairing. For example, the methane fraction in <code>premixed</code> is 8%, and the tuple is formatted as <b>("CH4", 0.08)</b>. Add all the non-zero components one by one to the <code>mixture_recipe</code> list as shown below. Once the <i>recipe</i> is set, simply pass it to the mixture. Here the fraction values given in <code>mixture_recipe</code> are mole fractions so you assign the <code>mixture_recipe</code> to <code>premixed</code> as <b>mole fractions</b>: <code>premixed.<b>X</b></code>. Similarly, if the fraction values are <b>mass fractions</b>, use <code>premixed.<b>Y</b></code>. 

In [ ]:
# molar composition of the mixture
mixture_recipe = [("CH4", 0.08), ("N2", 0.6), ("O2", 0.2), ("H2O", 0.12)]
# set mixture mole fractions
premixed.X = mixture_recipe

You can now find the <u>mean molecular weight</u> of the <code>premixed</code> mixture.

In [ ]:
# find the mixture mean molecular mass
print(f"mean molecular mass = {premixed.WTM:f} gm/mole")

Or convert between mass and mole fractions by switching the attribute from <b>Y</b> and <b>X</b>. 

In [ ]:
# convert automatically to mass fractions
print("mixture mass fractions (raw data):")
print(str(premixed.Y))
# switch back to mole fractions
print("\nmixture mole fractions (raw data):")
print(str(premixed.X))

List the non-zero components of <code>premixe</code> mixture in <u>mole fraction</u>. Use <code>mode="mass"</code> to display <u>mass fractions</u>. 

In [ ]:
print("\nformatted mixture composition output:")
premixed.listcomposition(mode="mole")

Once a <em>Mixture</em> object is created, you can make an identical mixture by doing a <i>hard</i> copy (<code>copy.deepcopy</code>).

In [ ]:
# create a "hard" copy of the premixed mixture ("soft" copying, i.e., anotherpremixed = premixed, works, too)
anotherpremixed = copy.deepcopy(premixed)
# list the composition of the copied mixture
print("\nformatted mixture composition of the copied mixture:")
anotherpremixed.listcomposition(mode="mole")

In [ ]:
# delete the premixed and anotherpremixed mixture objects
del premixed, anotherpremixed

The <b><u>array</u></i> method is straight forward.

<h5>Step one</h5>Instantiate the <code>syngas</code> <em>Mixture</em>.

In [ ]:
# create a mixture called premixed based on the MyGasMech chemistry set
syngas = ck.Mixture(MyGasMech)

<h5>Step two</h5>Specify the species composition of <code>syngas</code> by setting up a mole fraction array

Make an empty double array <code>frac</code> with its size = number of species in <code>MyGasMech</code> (<code>MyGasMech.<b>KK</b></code>).

In [ ]:
# species mole fractions of added/inert mixture. can also create an additives mixture here
frac = np.zeros(MyGasMech.KK, dtype=np.double)

Now find the species indices of the non-zero components of the mixture <code>syngas</code>. 

In [ ]:
# carbon monoxide CO species index
CO_index = MyGasMech.getspecindex("CO")
# hydrogen H2 species index
H2_index = MyGasMech.getspecindex("H2")
# methane CH4 species index
CH4_index = MyGasMech.getspecindex("CH4")
# carbon dioxide CO2 species index
CO2_index = MyGasMech.getspecindex("CO2")
# water H2O species index
H2O_index = MyGasMech.getspecindex("H2O")

Provide the mole fraction values of those non-zero components of <code>syngas</code> mixture.  

In [ ]:
# define molar compositions
frac[CO_index] = 0.5
frac[H2_index] = 0.25
frac[CH4_index] = 0.05
frac[CO2_index] = 0.1
frac[H2O_index] = 0.1

<h5>Step three</h5>Assign the <code>frac</code> array as the mole fractions to <code>syngas</code>.

In [ ]:
# set mixture mole fractions
syngas.X = frac

Display the non-zero components of <code>syngas</code> in <u>mole fractions</u> for verification.

In [ ]:
syngas.listcomposition(mode="mole")

In [ ]:

# delete the temporary array frac and syngas mixture object
del frac, syngas

A more complicated way to create a <em>Mixture</em> object in PyChemkin is to use the <i><u>equivalence ratio</u></i> method. It involves creating a <b><i>fuel</i> mixture</b>, an <b><i>oxidizer</i> mixture</b>, a <b>list of <u>complete combustion products</u></b>, and an optional <b><u>additives</u> species composition array/recipe</b>.  

<h5>Step one</h5>Create the <code>fuel</code> and the <code>air</code> mixtures. You can use either the <i><u>recipe</u></i> method or the <u><i>array</u></i> method.  

In [ ]:
# create a premixed fuel-oxidizer mixture by assigning the equivalence ratio
# create the fuel mixture
fuel = ck.Mixture(MyGasMech)
# set fuel composition
fuel.X = [("CH4", 1.0)]
# setting pressure and temperature is not required in this case
fuel.pressure = 5.0 * ck.Patm
fuel.temperature = 1500.0
# create the oxidizer mixture: air
air = ck.Mixture(MyGasMech)
air.X = [("O2", 0.21), ("N2", 0.79)]
# setting pressure and temperature is not required in this case
air.pressure = 5.0 * ck.Patm
air.temperature = 1500.0

<h5>Step two</h5>Make a list <code>products</code> for the <b><u>complete combustion products</u></b>. For hydrocarbons combustion in air, the complete combustion products are water (H2O), carbon dioxide (CO2), and nitrogen (N2).  

In [ ]:
# products from the complete combustion of the fuel mixture and air
products = ["H2O", "CO2", "N2"]

<h5>Step three</h5>Ddefine the <b><u>additives</u></b>. You can use an array (of mass/mole fractions) or an <b><u>addtives</u></b> mixture object. Here the <code>add_frac</code> is created with all zero values to indicate that there is no additive species.  

In [ ]:
# species mole fractions of added/inert mixture. can also create an additives mixture here
add_frac = np.zeros(MyGasMech.KK, dtype=np.double)   # no additives: all zeros

<h5>Step four</h5>Instantiate the target mixture: <code>premixed</code> <em>Mixture</em> object.

In [ ]:
# create the premixed mixture to be defined
premixed = ck.Mixture(MyGasMech)

<h5>Step five</h5>With all the required preparation tasks completed, you can set up the stoichiometric <code>premixed</code> mixture by setting the <code>equivalenceratio</code> value to 1. Use <code><b>X</b>byEquivalenceratio</code> when the fractions of <code>fuel</code>, <code>air</code>, and <code>add_frac</code> are in <u>mole fractions</u>; likewise, use <code><b>Y</b>byEquivalenceRatio</code> for <u>mass fractions</u>. 

In [ ]:
iError = premixed.XbyEquivalenceRatio(MyGasMech, fuel.X, air.X, add_frac, products, equivalenceratio=1.0)
if iError > 0:
    raise RuntimeError

List the non-zero components to verify the mixture composition.

In [ ]:
# list the composition of the premixed mixture
premixed.listcomposition(mode="mole")

In [ ]:
# delete all objects
del premixed, fuel, air, products, add_frac

This tutorial shows various ways to create a <em>Mixture</em> object in <strong>PyChemkin</strong>. Once the <em>Mixture</em> is set up, you can extract species amd mixture properties with various <em>Mixture</em> methods or manipulate the mixtures to form new ones.  